In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read Data

In [ ]:
df=pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

# Explore Data

In [ ]:
df.head()

In [ ]:
df.shape

Dataset contains 12 features including Target Variable Quality

In [ ]:
#let us check the data type for features
df.info()

All features are either Int or float data type

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
#There are 240 duplicate entries out of 1599

In [ ]:
df=df.drop_duplicates(keep='first')

In [ ]:
df.shape

There is no null value and duplicates have been removed

# Eploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.columns

In [ ]:
#Univariated Data Analysis
df['quality'].value_counts()

Quality above 6.5 is good quality wine

In [ ]:
sns.countplot(x=df['quality'])

In [ ]:
sns.pairplot(df, hue='quality')

# Remove outliers

In [ ]:
plt.figure(figsize=(40,20))
sns.boxplot(data=df)


In [ ]:
q1=df.quantile(0.25)
q2=df.quantile(0.75)
IQR=q2-q1

In [ ]:
df=df[~((df<(q1-1.5*IQR))|(df>(q2+1.5*IQR))).any(axis=1)]

Dataset after outlier removal

In [ ]:
plt.figure(figsize=(40,20))
sns.boxplot(data=df)

# Standradise & Label Encoding

In [ ]:
def cat(x):
    if x>6.5:
        return 'good'
    else:
        return 'poor'
    

In [ ]:
df['rating']=df['quality'].apply(lambda x: cat(x))

In [ ]:
X=df.drop(labels=['quality','rating'],axis=1)
y=df['rating']

In [ ]:
from sklearn.preprocessing import LabelEncoder
encode=LabelEncoder()
y_encode=encode.fit_transform(y)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=scaler.fit_transform(X)

# Train & Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_encode, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
def fit_predict(model1):
    model = model1
    model.fit(X_train, y_train)
    y_pred=model.predict(X_test)
    print("Accuracy score: {}".format(accuracy_score(y_test, y_pred)))
    print("classification report\n")
    print(classification_report(y_test,y_pred))
    print("\n")
    sns.heatmap(confusion_matrix(y_test,y_pred), annot=True)
    print("\n")
    print("{} model has accuracy of {}".format(str(model),accuracy_score(y_test, y_pred)))
    return model

# Logistic regression

In [ ]:
fit_predict(LogisticRegression())

# KNeighbourClassifier

In [ ]:
fit_predict(KNeighborsClassifier(n_neighbors=2))

In [ ]:
from sklearn.svm import SVC
fit_predict(SVC())

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
fit_predict(DecisionTreeClassifier())

In [ ]:
from sklearn import tree
model=fit_predict(DecisionTreeClassifier())


In [ ]:
plt.figure(figsize=(40,60))
tree.plot_tree(model)